<a href="https://colab.research.google.com/github/promptmule4real/demo/blob/main/promptmule_reporting_api_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PromptMule Reporting API Demo v0.2

Welcome to our interactive demonstration guide. Using the PromptMule cache enables you to automatically cache prompts and responses to OpenAI, without having to manage those yourself. These will be kept in the AWS Cloud until you wish to download them via a the Report API. First, you will need to generate some traffic with a few prompts, then some reports.

This tutorial will guide you through the process of using the Reporting API, enabling you to harness the full potential of the PromptMule reporting service. Tailored specifically for a Google Colaboratory runtime environment, this demo provides an engaging exploration into sending reports through OpenAI and testing reporting functions. If you do not have an API-KEY or Token, see the Github guide to generate those here:[https://github.com/promptmule4real/demo/blob/main/promptmule_api_demo.ipynb]

If you are reading this while on Github(https://github.com/): Start by clicking the "Open in Colab" icon located at the top left corner of the Github Preview page. This action will open the Google Colab notebook in a new tab, giving you the ability to run, modify, and interact with the demo code within.

Once you've launched the notebook in Google Colab, follow the subsequent step-by-step instructions below to effectively use the PromptMule Reporting API.

Stay tuned for a more comprehensive demo of other PromptMule functionalities, set to debut at the end of our next sprint!

We invite you to dive into this interactive exploration, and welcome your participation and feedback. Please visit us at www.promptmule.com.

If you are reading this while in Google Colab(https://colab.research.google.com/): To start, simply click on the "Run Cell" button located to the left of the code block. It resembles the play button you might find on a media player. Once activated, feel free to explore and experiment with the code.

In [ ]:
import os
import requests
import json
import textwrap

In [ ]:
# @title Input your PromptMule Sign-in Attributes { run: "auto", display-mode: "both" }
#@markdown Below replace the capital LETTERS with your information, pick a username, complex password, real email, and an application name that you will use as a reference in the future.

your_username = "YOUR_USERNAME"  #@param {type: "string"}  # this is your username, it will need to be unique, else the API will tell you it's not and try again (str)
api_key = "YOUR_PROMPTMULE_API_KEY"    #@param {type: "string"} # complex passwords are required (str)
your_appname = "YOUR_APPNAME"    #@param {type: "string"}   # this is the application that you are using the promptmule API to build, this app name is unique (str)
api_token = "YOUR_PROMPTMULE_TOKEN"    #@param {type: "string"}   # this is the application that you are using the promptmule API to build, this app name is unique (str)
#@markdown ---

In [ ]:
# @title Input OpenAI API KEY { run: "auto", display-mode: "form" }
#@markdown Input your OpenAI API key here. To obtain an OpenAI API key (https://platform.openai.com/account/api-keys), OR sign up on the OpenAI website, provide necessary information, and upon approval, you'll be issued an API key to authenticate your requests to the API.

OPENAI_API_KEY = 'YOUR_OPENAI_API_KEY' #@param {type: "string"} # this is your OpenAI API Key, it may impact your model choice
#@markdown ---


# PromptMule's /prompt endpoint API

Using the PromptMule cache enables you to automatically cache prompts and responses to OpenAI, without having to manage those yourself. These will be kept in the AWS Cloud until you wish to download them via a the Report API. First, you will need to generate some traffic with a few prompts.

You can utilize the /prompt endpoint using your PromptMule Token and API key, along with your OPENAI_API_KEY. These can be used to interact, via the PromptMule cache, with GPT-4 and/or GPT-3.5 models via the chat completion endpoint of OpenAI, available at: https://api.openai.com/v1/chat/completions

For information on how to structure the OpenAI Request Body, refer to the official documentation here: https://platform.openai.com/docs/api-reference/chat/create

We have incorporated some unique parameters specific to `promptmule`:

- `api` (String): A required field, for this demonstration use "openai" which is the only supported API currently.

- `semantic` (Float): This required parameter specifies the "percentage match" of the sent prompt with existing prompts in the cache. If a match is found with a percentage equal to or higher than this value, a maximum number of matches (as defined by `sem_num`) will be returned in a JSON dictionary.

- `sem_num` (Integer): A required parameter which should be a number between 1 and 10. It represents the maximum number of semantic matches to be found in the cache and returned in the API response as a JSON dictionary.

We need to update the headers for the Prompt API call, per below with Authorization which is different than previous calls using your API Token, and x-api-key which is your PromptMule API-Key, and your OpenAI API-Key.

In [ ]:
ENDPOINT = 'https://820czjhki0.execute-api.us-west-2.amazonaws.com/dev/'
PROMPT = 'prompt'

headers = {           # this is the /prompt endpoint header
    'Authorization': api_token,
    'x-api-key': api_key,
    'openai-key': OPENAI_API_KEY,
    'Content-Type': 'application/json'
    }

api_call_body = {     # this is a /prompt call body example
    "model": "gpt-4-0613",  # using GPT-4, you must have access to this model, or the call will fail
    "messages": [
        {
            "role": "user",
            "content": "Create a Python function for the chances of seeing a unicorn in Central Park. Assume the probability is the square root of -1."
        }
    ],
    "max_tokens": "50",
    "temperature": "0.99",
    "top_p": "1",
    "n": "1",
    "logprobs": "null",
    "stop": "null",
    "suffix": "null",
    "echo": "true",
    "presence_penalty": "0",
    "frequency_penalty": "0",
    "best_of": "1",
    "logit_bias": "null",
    "user": your_new_username,
    "api": "openai",    # this is used to denote openai as the destination
    "semantic": "0.99", # this denotes the semantic match percentage in the cache
    "sem_num": "2"      # this denotes the number of matches to return if found
}

Note that we are pointing the endpoint at the Prompt endpoint this time, and then sending the API call.

In [ ]:
promptmule = ENDPOINT + PROMPT
promptmule_response = requests.request("POST", promptmule, headers=headers, json=api_call_body)
# Handle response
if promptmule_response.status_code == 200:
    print("Response from OpenAI successful:\n", json.dumps(json.loads(promptmule_response.text), indent=4))
else:
    print(f"Response from OpenAI failed with status code {promptmule_response.status_code}:\n", json.dumps(json.loads(promptmule_response.text), indent=4))

In [ ]:
# Run the loop 10 times
for _ in range(10):
    try:
        promptmule = ENDPOINT + PROMPT
        promptmule_response = requests.request("POST", promptmule, headers=headers, json=api_call_body)
        response_json = json.loads(promptmule_response.text)

        # Handle response
        if promptmule_response.status_code == 200:
            print("Response from OpenAI successful:\n", json.dumps(response_json, indent=4))
        elif promptmule_response.status_code == 400:
            print(f"Bad request. Details: {response_json}")
        elif promptmule_response.status_code == 500:
            print(f"Runtime Service Exception. Details: {response_json}")
        else:
            print(f"Unexpected status code {promptmule_response.status_code}. Details: {response_json}")

    except requests.exceptions.HTTPError as errh:
        print(f"Http Error:{errh}")

    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting:{errc}")

    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error:{errt}")

    except requests.exceptions.RequestException as err:
        print(f"Something went wrong with the request:{err}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [ ]:
# This is the Application Use Report API based on a user's API KEY
USAGE = "/usage" # this is the Applicaton Use Report endpoint

headers = {           # this is the /usage endpoint header
    'Authorization': api_token,
    'x-api-key': api_key,
    'Content-Type': 'application/json'
    }


In [ ]:
#Run the Application Usage Report for this API KEY
try:
    promptmule = ENDPOINT + USAGE
    promptmule_response = requests.get(promptmule, headers=headers)
    response_json = json.loads(promptmule_response.text)

    # Handle response
    if promptmule_response.status_code == 200:
        print("Response from API successful:\n", json.dumps(response_json, indent=4))
    elif promptmule_response.status_code == 400:
        print(f"Bad request. Details: {response_json}")
    elif promptmule_response.status_code == 500:
        print(f"Runtime Service Exception. Details: {response_json}")
    else:
        print(f"Unexpected status code {promptmule_response.status_code}. Details: {response_json}")

except requests.exceptions.HTTPError as errh:
    print(f"Http Error:{errh}")

except requests.exceptions.ConnectionError as errc:
    print(f"Error Connecting:{errc}")

except requests.exceptions.Timeout as errt:
    print(f"Timeout Error:{errt}")

except requests.exceptions.RequestException as err:
    print(f"Something went wrong with the request:{err}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# This is the header setup for the Application Use Report API based on a user's API KEY
DATE_RANGE_REPORT = "/usage/daily-stats" # this is the Applicaton Use Report endpoint

headers = {           # this is the /usage endpoint header
    'Authorization': api_token,
    'x-api-key': api_key,
    'Content-Type': 'application/json'
    }

In [ ]:
# Run the Date Range Report for Application per API KEY
try:
    promptmule = ENDPOINT + DATE_RANGE_REPORT
    promptmule_response = requests.get(promptmule, headers=headers)
    response_json = json.loads(promptmule_response.text)

    # Handle response
    if promptmule_response.status_code == 200:
        print("Response from API successful:\n", json.dumps(response_json, indent=4))
    elif promptmule_response.status_code == 400:
        print(f"Bad request. Details: {response_json}")
    elif promptmule_response.status_code == 500:
        print(f"Runtime Service Exception. Details: {response_json}")
    else:
        print(f"Unexpected status code {promptmule_response.status_code}. Details: {response_json}")

except requests.exceptions.HTTPError as errh:
    print(f"Http Error:{errh}")

except requests.exceptions.ConnectionError as errc:
    print(f"Error Connecting:{errc}")

except requests.exceptions.Timeout as errt:
    print(f"Timeout Error:{errt}")

except requests.exceptions.RequestException as err:
    print(f"Something went wrong with the request:{err}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

NameError: ignored

# Retreive all API Keys for a username

In [ ]:
# This is the header setup for the Get All User API Keys
GET_ALL_API_KEYS_PER_USER = "/api-keys" # this is the Get ALL User API Keys endpoint

headers = {           # this is the GET /api-keys endpoint header
    'Authorization': api_token,
    'Content-Type': 'application/json'
    }

In [ ]:
# Run the Date Range Report for Application per API KEY
try:
    promptmule = ENDPOINT + GET_ALL_API_KEYS_PER_USER
    promptmule_response = requests.get(promptmule, headers=headers)
    response_json = json.loads(promptmule_response.text)

    # Handle response
    if promptmule_response.status_code == 200:
        print("Get All Keys for User successful:\n", json.dumps(response_json, indent=4))
    elif promptmule_response.status_code == 400:
        print(f"Bad request. Details: {response_json}")
    elif promptmule_response.status_code == 500:
        print(f"Runtime Service Exception. Details: {response_json}")
    else:
        print(f"Unexpected status code {promptmule_response.status_code}. Details: {response_json}")

except requests.exceptions.HTTPError as errh:
    print(f"Http Error:{errh}")

except requests.exceptions.ConnectionError as errc:
    print(f"Error Connecting:{errc}")

except requests.exceptions.Timeout as errt:
    print(f"Timeout Error:{errt}")

except requests.exceptions.RequestException as err:
    print(f"Something went wrong with the request:{err}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# This is the header setup for the Get All User Prompts
PROMPT = "/prompt" # this is the Get ALL User API Keys endpoint

headers = {           # this is the GET /api-keys endpoint header
    'Authorization': api_token,
    'x-api-key': api_key,
    'Content-Type': 'application/json'
    }
# NOTE the use of parameters here to convey the start date of the prompts you want to retrieve and the end date. Also, you can choose to ask for prompts that have been cached or not, this is done with the "is_cached" parameter, where 'TRUE' means that the values will be returned form the live cache, and 'FALSE' returns any prompt that has occured between the two dates. Finally, 'limit' is the maximum number (int) between 1 and 100 of the prompts to return in the reply json.
params = {
    'start-date': '2023-07-01',
    'end-date': '2023-07-27',
    'is-cached': 'False',
    'limit': '10'
}

In [ ]:
# Run the Date Range Report for Application per API KEY
try:
    promptmule = ENDPOINT + PROMPT
    promptmule_response = requests.get(ENDPOINT, headers=headers, params=params)
    response_json = json.loads(promptmule_response.text)

    # Handle response
    if promptmule_response.status_code == 200:
        print("The prompts were found successfully:\n", json.dumps(response_json, indent=4))
    elif promptmule_response.status_code == 400:
        print(f"Bad request. Details: {response_json}")
    elif promptmule_response.status_code == 500:
        print(f"Runtime Service Exception. Details: {response_json}")
    else:
        print(f"Unexpected status code {promptmule_response.status_code}. Details: {response_json}")

except requests.exceptions.HTTPError as errh:
    print(f"Http Error:{errh}")

except requests.exceptions.ConnectionError as errc:
    print(f"Error Connecting:{errc}")

except requests.exceptions.Timeout as errt:
    print(f"Timeout Error:{errt}")

except requests.exceptions.RequestException as err:
    print(f"Something went wrong with the request:{err}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


## Congratulations! You have are one of us now!
That's the end of this demo. Let us know what you think.

## Handling Errors in Colab or PromptMule API

In the course of running the demo, should you encounter any errors or issues, the Colab environment provides you with features to debug and resolve them.

1. **Runtime Reset**: If your notebook's runtime seems stuck or producing unexpected errors, you can reset it by selecting 'Runtime > Restart runtime...' from the menu.

2. **Check Error Logs**: Colab provides detailed error logs for each cell. If a cell execution fails, you will see an error message below the cell which can provide clues on what went wrong.

3. **PromptMule Support can reset your username**: And your email/passwrod/Token/Key if needed. Reach out to us and join our private preview slack at support@promptmule.com